In [1]:
from sklearn.preprocessing import StandardScaler
import numpy as np
import pandas as pd

import tensorflow as tf
from SMDF.logging import logger

In [22]:
import joblib
import datetime

In [2]:
pwd

'd:\\Full Stack Data Science\\Time Series Analysis\\MAJOR PROJECT\\SMDF\\research'

In [3]:
cd ..

d:\Full Stack Data Science\Time Series Analysis\MAJOR PROJECT\SMDF


In [4]:
from pathlib import Path

In [23]:
class ForecastingPipeline:
    def __init__(self):
        self.M01AB = tf.saved_model.load(Path("artifacts/model_trainer/M01AB"))
        self.M01AE = tf.saved_model.load("artifacts/model_trainer/M01AE")
        self.N02BA = tf.saved_model.load("artifacts/model_trainer/N02BA")
        self.N02BE = tf.saved_model.load("artifacts/model_trainer/N02BE")
        self.N05B = tf.saved_model.load("artifacts/model_trainer/N05B")
        self.N05C = tf.saved_model.load("artifacts/model_trainer/N05C")
        self.R03 = tf.saved_model.load("artifacts/model_trainer/R03")
        self.R06 = tf.saved_model.load("artifacts/model_trainer/R06")
        self.train = pd.read_csv("artifacts/data_spliting/train.csv")
        self.test = pd.read_csv("artifacts/data_spliting/test.csv")
        self.scaler = joblib

    def noramalizing_data(self,column_name):
        scaler = StandardScaler()
        #self.train.drop(columns=["datum","Year","Month","Hour","Weekday Name"],inplace=True)
        #self.test.drop(columns=["datum","Year","Month","Hour","Weekday Name"],inplace=True)

        train_data = scaler.fit_transform(self.train[column_name].values.reshape(-1,1))

        test_data = scaler.transform(self.test[column_name].values.reshape(-1,1))

        
        logger.info(test_data.shape)
        
        #print(test_data)
        return  test_data,train_data
    
    def test_spliting(self,column_name):
        train, test = self.noramalizing_data(column_name)
        window_size = 60
        # Concatenate train data to test data
        dataset_total = np.concatenate((train, test), axis = 0)
        # Split test data and last window-size of train data
        inputs = dataset_total[len(dataset_total) - len(test) - window_size:]
        # Do the same thing for test data
        X_test = []
        y_test = []
        for i in range(window_size, window_size+len(test)):
            X_test.append(inputs[i-window_size:i,:]) 
            y_test.append(inputs[i,-1]) # consider Close as target
        # Change them to numpy array
        X_test, y_test = np.array(X_test).astype('float32'), np.array(y_test).reshape(-1, 1)
        logger.info(X_test.shape)
    
        return X_test,y_test


    def helper_forecast(self,column_name,key):
        X_test, _ = self.test_spliting(column_name)
        
        logger.info("Forecasting Start ")
        logger.info(X_test.shape)
        models = [self.M01AB,
                  self.M01AE,
                  self.N02BA,
                  self.N02BE,
                  self.N05B,
                  self.N05C,
                  self.R03,
                  self.R06]
        time_distribution = ['time_distributed_1',
                             'time_distributed_3',
                             'time_distributed_5',
                             'time_distributed_7',
                             'time_distributed_9',
                             'time_distributed_11',
                             'time_distributed_13',
                             'time_distributed_15']
        
        x_input=X_test[-1:].reshape(1,-1)
        
        
        temp_input=list(x_input)
        temp_input=temp_input[0].tolist()
        logger.info(len(temp_input))
        
    
        lst_output = []
        n_step = 60
        i = 0
        while i<30:
        
            if len(temp_input) >60:
                x_input=np.array(temp_input[1:])
                x_input=x_input.reshape(1,-1)
                x_input = x_input.reshape((1, n_step, 1))
                #print(x_input)
                #forecast.append(x_input)
                #print(forecast)
                #logger.info("60 >")
                x_input = tf.constant(x_input, dtype=tf.float32)
                model = models[key].signatures["serving_default"]
                test_predict = model(x_input)
                #logger.info(test_predict)
                #print(test_predict)
                predictions = test_predict[time_distribution[key]].numpy()
                temp_input=temp_input[1:]
                lst_output.extend(predictions[0][0].tolist())
                i=i+1

            else:
                x_input = np.array(x_input)
                x_input = x_input.reshape((1, n_step,1))
                
                #x_input = x_input.astype(float) 
                model = models[key].signatures["serving_default"]
                test_predict = model(tf.constant(x_input))
                #logger.info(test_predict)
                #print(test_predict)
                predictions = test_predict[time_distribution[key]].numpy()
                #yhat = model.predict(x_input, verbose=0)
                temp_input.extend(predictions[0][0].tolist())
                lst_output.extend(predictions[0][0].tolist())
                i = i+1
                #logger.info(type(temp_input[0]))
                    
       

        
        return lst_output
    def forecasting(self):
        
        today = datetime.datetime.now().strftime('%m/%d/%Y')
        forecast_dates=pd.date_range(start=today, periods = 30)
        forecast=pd.DataFrame(forecast_dates,columns=['Date'])
        forecast['Date']=pd.to_datetime(forecast['Date'])
        forecast.sort_values('Date',inplace=True)
        columns = ["M01AB",
            "M01AE",
            "N02BA",
            "N02BE",
            "N05B",
            "N05C",
            "R03",
            "R06"]
        for i,val in enumerate(columns):
            forecast_data = self.helper_forecast(val,i)
            #forecast_data=self.scaler.inverse_transform(np.array(forecas_data).reshape(-1, 1))
            forecast[val] = forecast_data
            
        return forecast.to_csv("artifacts/forecast_data/forecast.csv",index=False),forecast
    
                
        
    

In [24]:
forecast_cls = ForecastingPipeline()
forecast_cls.forecasting()


[2023-07-02 21:02:50,604: INFO: 1444572149: (422, 1)]
[2023-07-02 21:02:50,612: INFO: 1444572149: (1684, 60, 1)]
[2023-07-02 21:02:50,612: INFO: 1444572149: Forecasting Start ]
[2023-07-02 21:02:50,620: INFO: 1444572149: (1684, 60, 1)]
[2023-07-02 21:02:50,620: INFO: 1444572149: 60]
[2023-07-02 21:02:51,069: INFO: 1444572149: (422, 1)]
[2023-07-02 21:02:51,086: INFO: 1444572149: (1684, 60, 1)]
[2023-07-02 21:02:51,094: INFO: 1444572149: Forecasting Start ]
[2023-07-02 21:02:51,102: INFO: 1444572149: (1684, 60, 1)]
[2023-07-02 21:02:51,110: INFO: 1444572149: 60]
[2023-07-02 21:02:51,470: INFO: 1444572149: (422, 1)]
[2023-07-02 21:02:51,478: INFO: 1444572149: (1684, 60, 1)]
[2023-07-02 21:02:51,478: INFO: 1444572149: Forecasting Start ]
[2023-07-02 21:02:51,486: INFO: 1444572149: (1684, 60, 1)]
[2023-07-02 21:02:51,486: INFO: 1444572149: 60]
[2023-07-02 21:02:51,837: INFO: 1444572149: (422, 1)]
[2023-07-02 21:02:51,861: INFO: 1444572149: (1684, 60, 1)]
[2023-07-02 21:02:51,861: INFO: 144

(None,
          Date     M01AB     M01AE     N02BA     N02BE      N05B      N05C  \
 0  2023-07-02  2.223779  0.535053  0.847471 -0.885573 -0.489597 -0.695162   
 1  2023-07-03  1.180167  0.249070  0.230377 -0.910907  1.368709 -0.419947   
 2  2023-07-04  1.180167  0.249070  0.230377 -0.910907  1.368709 -0.419947   
 3  2023-07-05  0.232661 -0.123547 -0.298821 -0.653181 -0.456261  0.012610   
 4  2023-07-06  0.232661 -0.123547 -0.298821 -0.653181 -0.456261  0.012610   
 5  2023-07-07 -1.560967 -0.962537 -0.447988 -0.647456 -0.025767 -0.026324   
 6  2023-07-08 -1.560967 -0.962537 -0.447988 -0.647456 -0.025767 -0.026324   
 7  2023-07-09  1.078282 -0.279994 -1.101143 -0.272654 -0.460058  0.503143   
 8  2023-07-10  1.078282 -0.279994 -1.101143 -0.272654 -0.460058  0.503143   
 9  2023-07-11 -0.175109 -0.120284 -0.704017 -0.441894 -0.344218 -0.440330   
 10 2023-07-12 -0.175109 -0.120284 -0.704017 -0.441894 -0.344218 -0.440330   
 11 2023-07-13 -0.568018  1.269271 -1.486926 -0.339246 -0